In [ ]:
# !pip install pyperclip
# !pip install selenium

In [4]:
from selenium import webdriver
import time
import pandas as pd
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pyperclip

### driver open


In [ ]:
driver = webdriver.Chrome('chromedriver.exe')  # 윈도우
time.sleep(1)

In [8]:
#드라이버 로딩 
driver = webdriver.Chrome('./chromedriver.exe') 

# ##사용할 변수 선언 
# #네이버 로그인 주소 
url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com' 
uid = 

#네이버 로그인 페이지로 이동 
driver.get(url) 
time.sleep(2) #로딩 대기 

#아이디 입력폼 
tag_id = driver.find_element_by_name('id') 
#패스워드 입력폼 
tag_pw = driver.find_element_by_name('pw')

# id 입력 
# 입력폼 클릭 -> paperclip에 선언한 uid 내용 복사 -> 붙여넣기 
tag_id.click() 
pyperclip.copy(uid) 
tag_id.send_keys(Keys.CONTROL, 'v') 
time.sleep(1) 

# pw 입력 # 입력폼 클릭 -> paperclip에 선언한 upw 내용 복사 -> 붙여넣기
tag_pw.click() 
pyperclip.copy(upw) 
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1) 

#로그인 버튼 클릭 
login_btn = driver.find_element_by_id('log.login') 
login_btn.click() 
time.sleep(2)



In [21]:
# a =  'https://cafe.naver.com/jejutip'
# driver.switch_to.frame("cafe_main")

In [6]:
driver.get('https://cafe.naver.com/jejutip')
tag_search = driver.find_element_by_name('query').send_keys('전기차')
driver.find_element_by_name("query").send_keys(Keys.ENTER)
time.sleep(2)
driver.switch_to.frame("cafe_main")

In [111]:

# driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/table/tbody/tr')[0].get_attribute("href")

In [ ]:
#main-area > div.article-board.result-board.m-tcol-c > table > tbody > tr:nth-child(5) > td.td_article > div.board-list > div > a.article

In [5]:
a = []

for i in range(1,16):
    a.append(driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]')[0].get_attribute("href"))
    
len(a)
    

15

In [ ]:
# address = []
# page = 0
# for i in range(1,16):
#     a = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]')[0].get_attribute("href")
#     address.append(a)
    
# address

In [7]:

address = []
page = 0

while page > -1:
    
    if page > 101:
        break
    
    try:
        page += 1
        driver.get(f"https://cafe.naver.com/jejutip?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10456178%26search.media=0%26search.searchdate=all%26userDisplay=15%26search.option=0%26search.sortBy=date%26search.searchBy=0%26search.query=%C0%FC%B1%E2%C2%F7%26search.viewtype=title%26search.page={page}")
        driver.switch_to_frame('cafe_main')
        try: 
            if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
                break
        except:
            pass
    
        for i in range(1,16):
            address.append(driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]')[0].get_attribute("href"))
            
    except:
        pass
    


C:\Users\pc\AppData\Local\Temp/ipykernel_14364/1259453112.py:12: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('cafe_main')


In [11]:
len(address)

1491

In [24]:
address[0]

'https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=10456178&page=1&inCafeSearch=true&searchBy=0&query=%EC%A0%84%EA%B8%B0%EC%B0%A8&includeAll=&exclude=&include=&exact=&searchdate=all&media=0&sortBy=date&articleid=2066007&referrerAllArticles=true'

In [18]:
driver.get(address[0])

In [ ]:
/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div/div/div/div/p

In [33]:
driver.get(address[1])
time.sleep(2)
driver.switch_to_frame('cafe_main')
title =driver.find_element_by_class_name("title_text").text
contents =driver.find_elements_by_tag_name('p')
content = ''
for i,item in enumerate(contents):
    content += contents[i].text+'\n'
print(title)
print(content)

C:\Users\pc\AppData\Local\Temp/ipykernel_14364/2837309640.py:3: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('cafe_main')


전기차 vs 휘발유 아반뗴? 뭐가 나을까요??


기름값이며 뭐며 하면 전기차가 나을 거 같은데
만약 사고날시 전기차가 엄청 비싸다고 얘기 들어서요ㅠㅠ
뭐가 나을까요??
충전도 그렇고 그냥 휘발유가 젤 편한거 같아요 아직은^^
전기차 차주인데 저는 휘발유했어요
전기차는 아직까지 불편해요...가는곳마다 충전해야한다는 불안함...
전기차는 충전해야하고 충전하는곳도 많지 않구요 휘발유하세요
렌트는 전기차 하지마세요
만약이라도 사고나면 몇천깨질수도 있어요
6세아들 전기차 빌려보자하는데..운전20년차에 전기차는 해본적없어서 매번 없다고 뻥치고 엘피지나 휘발유 빌립니다ㅋ
휘발유차요
휘발유 저렴한거, 없으면 LPG ..
저는 항상 이기준으로 렌트합니다
저도휘발유요
우연히 댓글보다가..휘발유 추천이 많네요..LPG렌트카는 요즘 별로 없나요?
clon522아~~~~!그렇군요. 궁금했는데..답변해주셔서 고맙습니다^^



In [17]:
titles = []
contentss = []

for i in range(len(address)):
    driver.get(address[i])
    time.sleep(2)
    
    driver.switch_to_frame('cafe_main')
    title =driver.find_element_by_class_name("title_text").text
    contents =driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/div[2]/div[1]/div/div/div/div/div/div/p')
    content = ''
    for i,item in enumerate(contents):
        content += contents[i].text+'\n'
    titles.append(title)
    contentss.append(content)    

    

C:\Users\pc\AppData\Local\Temp/ipykernel_14364/2779812529.py:7: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('cafe_main')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".title_text"}
  (Session info: chrome=99.0.4844.51)


In [ ]:
df=pd.DataFrame({"제목":titles ,"내용":contentss})
df.head(3)
